# LLM regression testing workflow

In [ ]:
#!pip install evidently[llm]

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
from io import BytesIO

To run open-source evaluations:

In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite
from evidently.metric_preset import TextEvals
from evidently.descriptors import *
from evidently.metrics import *
from evidently.tests import *
from evidently.features.llm_judge import BinaryClassificationPromptTemplate

**Optional**: To work with Evidently Cloud:

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace

**Optional**: To manage dashboards as code remotely. You can also do this in UI.

In [ ]:
from evidently.ui.dashboards import DashboardPanelTestSuite
from evidently.ui.dashboards import PanelValue
from evidently.ui.dashboards import ReportFilter
from evidently.ui.dashboards import TestFilter
from evidently.ui.dashboards import TestSuitePanelType
from evidently.renderers.html_widgets import WidgetSize

# Prepare a dataset

Get an example dataset. You can also download and import the CSV file directly ([Link](https://github.com/evidentlyai/evidently/blob/main/examples/how_to_questions/chat_df.csv)).

In [ ]:
response = requests.get("https://raw.githubusercontent.com/evidentlyai/evidently/main/examples/how_to_questions/chat_df.csv")
csv_content = BytesIO(response.content)

Read the CSV content into a DataFrame. Parse dates and set conversation "start_time" as index.

In [ ]:
assistant_logs = pd.read_csv(csv_content, index_col=0, parse_dates=['start_time', 'end_time'])
assistant_logs.index = assistant_logs.start_time
assistant_logs.index.rename('index', inplace=True)

Preview:

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
assistant_logs.head(3)

# Set up your Workspace 

To store the evaluation results, share them with others, and get a live monitoring dashboard, create your free Evidently Cloud account at: http://app.evidently.cloud/signup.

 To connect to the workspace from your Python environment:

In [ ]:
ws = CloudWorkspace(token="YOUR_TOKEN", 
                    url="https://app.evidently.cloud")

Create your Project:

In [ ]:
project = ws.create_project("My project title", team_id="YOUR_TEAM_ID")
project.description = "My project description"
project.save()

# Run evaluations

Prep: map your input data columns. Optional, but recommended.

In [ ]:
column_mapping = ColumnMapping(
    datetime='start_time',
    datetime_features=['end_time'],
    text_features=['question', 'response'],
    categorical_features=['organization', 'model_ID', 'region', 'environment', 'feedback'],
)

## Basic example

Run the first evaluation by checking the chatbot response length. You will use the `TextLength()` descriptor. This will return an absolute count for the number of symbols in each text. You can also check `SentenceCount()`, `WordCount()`, etc.

To run the evaluation for the first 100 conversations and get a summary Report:

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response",
              descriptors=[
                  TextLength(),
                  ]
              )
])

text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:100],
                      column_mapping=column_mapping)
text_evals_report

You can also do a side-by-side comparison for two datasets:

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response",
              descriptors=[
                  TextLength(),
                  ]
              )
])

text_evals_report.run(reference_data=assistant_logs[:50],
                      current_data=assistant_logs[50:100],
                      column_mapping=column_mapping)
text_evals_report

Let's look at other evaluation methods one by one. You can later combine multiple descriptors in a single Report.

## Text patterns

You can use regular expressions to check text patterns. For example, check the presence of competitor mentions, topical words, etc.

Let's check for responses that contain words related to compensation. This will automatically account for inflected and variant words. This descriptor returns True/False for **pattern match**.

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response",
              descriptors=[
                  IncludesWords(
                      words_list=['salary', 'benefits', 'payroll'],
                      display_name="Mention Compensation")
            ]
        ),
        ]
)

text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:100],
                      column_mapping=column_mapping)
text_evals_report

Other examples: `Contains(items=[])`, `BeginsWith(prefix="")`, custom `RegEx(reg_exp=r"")`, etc.

## Model-based scoring

You can use pre-trained machine learning models to score your text data.

**Sentiment**. You can use built-in models like `Sentiment()`. This will return a sentiment score from -1 (very negative) to 1 (very positive).

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
            Sentiment(),
        ]
    ),
])

text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:100],
                      column_mapping=column_mapping)
text_evals_report

You can also use models from HuggingFace. This will download the models to score your data locally.

**Toxicity**. You can use a pre-selected toxicity model using `HuggingFaceToxicityModel()` descriptor. This will returns the predicted toxicity score between 0 to 1.

**Neutral emotion**. You can call a named custom model from HuggingFace. For example, let's use the `SamLowe/roberta-base-go_emotions` model and get a score from 0 to 1 for "neutral" label to see if responses convey neutral emotion.

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
            HuggingFaceToxicityModel(),
            HuggingFaceModel(
                model="SamLowe/roberta-base-go_emotions",
                params={"label": "neutral"},
                display_name="Response Neutrality"),
        ]
    ),
])

text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:100],
                      column_mapping=column_mapping)
text_evals_report

See docs on using HuggingFace models as descriptors: https://docs.evidentlyai.com/user-guide/customization/huggingface_descriptor

## LLM-as-a-judge

You can use LLMs with custom evaluation prompts to label the texts by specific criteria such as tone or conciseness. You can use `OpenAIPrompting` descriptor.

This descriptor requires an OpenAI key. Pass it as an environment variable.

In [ ]:
## import os

## os.environ["OPENAI_API_KEY"] = "YOUR KEY"

In [ ]:
report = Report(metrics=[

    TextEvals(column_name="response", descriptors=[
        DeclineLLMEval(),
        PIILLMEval(include_reasoning=True), 
    ])
])

report.run(reference_data= None,
           current_data= assistant_logs[:10],
           column_mapping=column_mapping)
report 

In [ ]:
custom_judge = LLMEval(
    subcolumn="category",
    template = BinaryClassificationPromptTemplate(      
        criteria = """Conciseness refers to the quality of being brief and to the point, while still providing all necessary information.
            A concise response should:
            - Provide the necessary information without unnecessary details or repetition.
            - Be brief yet comprehensive enough to address the query.
            - Use simple and direct language to convey the message effectively.
        """,
        target_category="Conciseness",
        non_target_category="Ok",
        uncertainty="unknown",
        include_reasoning=True,
        pre_messages=[("system", "You are a judge which evaluates text.")],
        ),
    provider = "openai",
    model = "gpt-4o-mini"
)

report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
        custom_judge
    ])
])

report.run(reference_data= None,
           current_data= assistant_logs[:10],
           column_mapping=column_mapping)
report 

You can also run evals using two columns: e.g., context and response, request and response. Check the docs: https://docs.evidentlyai.com/user-guide/customization/llm_as_a_judge

## Metadata columns

To summarize metadata fields:

In [ ]:
data_report = Report(metrics=[
   ColumnSummaryMetric(column_name="feedback"),
   ]
)

data_report.run(reference_data=None, current_data=assistant_logs[:100], column_mapping=column_mapping)
data_report

## Semantic similarity

**Semantic similarity** evaluates how close two texts are in meaning using an embedding model. It returns a score from 0 to 1 (0: different, 0.5: unrelated, 1: similar). This is a pairwise descriptor. Let's compare the similarity between questions and responses:

In [ ]:
text_evals_report = Report(metrics=[
    ColumnSummaryMetric(
        column_name=SemanticSimilarity(
            display_name="Response-Question Similarity"
        )
        .on(["response", "question"])
    )
])

text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:100],
                      column_mapping=column_mapping)
text_evals_report

You can also run evals to compare current responses against "golden" reference examples. Check a tutorial on regression testing: https://www.evidentlyai.com/blog/llm-regression-testing-tutorial

# Export evaluation results

View the dataset with added evaluation results for each row:

In [ ]:
text_evals_report.datasets()[1]

Python dictionary with a summary report:

In [ ]:
text_evals_report.as_dict()

JSON with a summary report:

In [ ]:
text_evals_report.json()

Save as HTML file:

In [ ]:
text_evals_report.save_html("report.html")

# Monitoring results over time

Define a combined Report:

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
            Sentiment(),
            TextLength(),
            IncludesWords(words_list=['salary', 'benefits', 'payroll'],
                          display_name="Mention Compensation")

        ],
    ),
    ColumnSummaryMetric(column_name="feedback"),
    ColumnSummaryMetric(column_name="region"),
    ColumnSummaryMetric(column_name="organization"),
    ColumnSummaryMetric(column_name="model_ID"),
    ColumnSummaryMetric(column_name="environment"),
])

Run the Report on the first 50 rows:

In [ ]:
text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[:50],
                      column_mapping=column_mapping)

In [ ]:
project.id

Send evaluation results to Evidently Cloud.

In [ ]:
ws.add_report(project.id, text_evals_report, include_data=True)

**Cloud UI**. In the Evidently Cloud UI, Add a "Descriptors" tab and a "Columns" tab to create a dashboard that plots the metrics.

Let's imitate a few consecutive runs to evaluate batches of data as they come. Run and send several Reports, each time taking 50 rows.

In [ ]:
text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[50:100],
                      column_mapping=column_mapping)
ws.add_report(project.id, text_evals_report)

In [ ]:
text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[100:150],
                      column_mapping=column_mapping)
ws.add_report(project.id, text_evals_report)

In [ ]:
text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[150:200],
                      column_mapping=column_mapping)
ws.add_report(project.id, text_evals_report)

In [ ]:
text_evals_report.run(reference_data=None,
                      current_data=assistant_logs[200:250],
                      column_mapping=column_mapping)
ws.add_report(project.id, text_evals_report)

# Extra: Run conditional tests

You can monitor not only values but whether they comply with the conditions you define. For example:
* Average response sentiment should be positive.
* Response length should always be non-zero.
* The maximum response length should be 2000 symbols.
* The mean response length should be above 500 symbols.

Define the test suite:

In [ ]:
test_suite = TestSuite(tests=[
    TestColumnValueMean(column_name = Sentiment().on("response"), gte=0),
    TestColumnValueMin(column_name = TextLength().on("response"), gt=0),
    TestColumnValueMax(column_name = TextLength().on("response"), lte=2000),
    TestColumnValueMean(column_name = TextLength().on("response"), gt=500),
])

Imitate sending 5 reports in a row with 1 hour difference in timestamps:

In [ ]:
for i in range(5):
    test_suite.run(
        reference_data=None,
        current_data=assistant_logs.iloc[50 * i : 50 * (i + 1), :],
        column_mapping=column_mapping,
        timestamp=datetime.now() + timedelta(hours=i)
    )
    ws.add_test_suite(project.id, test_suite)

**Add a test suite panel**. Copy your Project ID to connect.

In [ ]:
#project = ws.get_project("YOUR_PROJECT_ID")

In [ ]:
project.dashboard.add_panel(
    DashboardPanelTestSuite(
        title="Test results",
        filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
        size=WidgetSize.FULL,
        panel_type=TestSuitePanelType.DETAILED,
    ),
    tab="Tests"
)
project.save()